In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# machine learning libs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 

#from sklearn.compose import ColumnTransformer, make_column_selector
#from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix

In [39]:
df = pd.read_csv("Documents/tree_based_ML_features_final.csv")
df

#data frame of just iphone 13

print(df[(df['phone_generation']==13)&(df['claim']<=500)])


      phone_generation  phone_size  week_of_year  month  quarter  year  \
6124                13         128            48     11        4  2021   
6125                13         128            50     12        4  2021   
6126                13         128            51     12        4  2021   
6127                13         128            52     12        4  2021   
6128                13         128             1      1        1  2022   
...                ...         ...           ...    ...      ...   ...   
7743                13         512             2      1        1  2023   
7744                13         512             3      1        1  2023   
7745                13         512             4      1        1  2023   
7746                13         512             5      1        1  2023   
7747                13         512             6      2        1  2023   

      holiday  days_since_release_date  claim  black  ...  apple iphone 6s  \
6124        0                    

In [40]:
#55 total
df.shape

(16579, 55)

In [41]:
df=df[['phone_generation','phone_size','week_of_year', 'month', 'quarter',
       'year', 'days_since_release_date', 'holiday','claim', 'black', 'blue',
       'coral', 'gold', 'graphite', 'gray', 'green', 'midnight', 'pink',
       'purple', 'red', 'silver', 'starlight', 'white', 'yellow']]

In [42]:
#only looking at these generations rather than the specific models for an iphone like pro, max, etc. because we are only looking to predict iphone 14
df['phone_generation'].unique()

array([11, 12, 13, 14,  3,  4,  5,  6,  7,  8, 10])

In [62]:
#split the data into train and test sets
from sklearn.model_selection import train_test_split

feature_selection = df.columns.to_list()
feature_selection.remove('claim')
feature_selection = df[feature_selection]

train_data = df[(df['phone_generation']==13)& (df['days_since_release_date'] <= 160)]
train_data=train_data[['days_since_release_date','phone_size','holiday','claim', 'black', 'blue',
       'coral', 'gold', 'graphite', 'gray', 'green', 'midnight', 'pink',
       'purple', 'red', 'silver', 'starlight', 'white', 'yellow']]
x_train= train_data.drop('claim', axis=1)
y_train = train_data['claim']

#val_data=df[(df['phone_generation'] == 13) & (df['days_since_release_date'] >=414) & (df['days_since_release_date'] <=470)]
#val_data=val_data[['days_since_release_date','phone_size','holiday','claim', 'black', 'blue',
       #'coral', 'gold', 'graphite', 'gray', 'green', 'midnight', 'pink',
       #'purple', 'red', 'silver', 'starlight', 'white', 'yellow']]
#x_val= val_data.drop('claim', axis=1)
#y_val = val_data['claim']

test_data=df[(df['phone_generation']==14) & (df['year'] >= 2023 ) & (df['month']==2) ]
test_data=test_data[['days_since_release_date','phone_size','holiday','claim', 'black', 'blue',
       'coral', 'gold', 'graphite', 'gray', 'green', 'midnight', 'pink',
       'purple', 'red', 'silver', 'starlight', 'white', 'yellow']]


features = test_data.drop('claim', axis=1)
target = test_data['claim']

In [63]:
print(train_data)

      days_since_release_date  phone_size  holiday  claim  black  blue  coral  \
6124                       36         128        0      2      0     1      0   
6125                       50         128        0      2      0     1      0   
6126                       57         128        1      1      0     1      0   
6127                       64         128        0      9      0     1      0   
6128                       71         128        0      3      0     1      0   
...                       ...         ...      ...    ...    ...   ...    ...   
7586                      127         256        0     13      0     0      0   
7587                      134         256        0     10      0     0      0   
7588                      141         256        0     13      0     0      0   
7589                      148         256        0     16      0     0      0   
7590                      155         256        0     10      0     0      0   

      gold  graphite  gray 

In [64]:
print(features)

      days_since_release_date  phone_size  holiday  black  blue  coral  gold  \
7760                      143         128        0      1     0      0     0   
7773                      143         128        0      0     0      0     0   
7785                      143         256        0      1     0      0     0   
7802                      143         256        0      0     0      0     0   
7805                      143         512        0      1     0      0     0   
7806                      143         512        0      0     0      0     0   
7807                      143         128        0      1     0      0     0   
7808                      143         128        0      0     0      0     1   
7809                      143         128        0      0     0      0     0   
7810                      143         128        0      0     0      0     0   

      graphite  gray  green  midnight  pink  purple  red  silver  starlight  \
7760         0     0      0         0   

In [65]:
#target is the claim value
print(target)

7760    31
7773    12
7785    11
7802     5
7805     1
7806     1
7807    71
7808    50
7809    68
7810     1
Name: claim, dtype: int64


In [66]:
# x train
print('On X train: ')
print('X train dimensions: ', x_train.shape)
display(x_train.head())
# x test
print('\nOn X test: ')
print('X test dimensions: ', x_test.shape)
display(x_test.head())

On X train: 
X train dimensions:  (73, 18)


,days_since_release_date,phone_size,holiday,black,blue,coral,gold,graphite,gray,green,midnight,pink,purple,red,silver,starlight,white,yellow
6124,36,128,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6125,50,128,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6126,57,128,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6127,64,128,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6128,71,128,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0



On X test: 
X test dimensions:  (26, 18)


,days_since_release_date,phone_size,holiday,black,blue,coral,gold,graphite,gray,green,midnight,pink,purple,red,silver,starlight,white,yellow
7755,108,128,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7756,115,128,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7757,122,128,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7758,129,128,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7759,136,128,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
# y train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())

# y test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())

On y train: 
y train dimensions:  (73,)


6124    2
6125    2
6126    1
6127    9
6128    3
Name: claim, dtype: int64


On y test: 
y test dimensions:  (26,)


7755    28
7756    37
7757    35
7758    39
7759    33
Name: claim, dtype: int64

In [68]:
np.unique(y_train)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        15,  16,  17,  19,  21,  22,  23,  24,  30,  31,  32,  35,  40,
        41,  43,  55,  56,  57,  61,  66,  69,  70,  71,  78,  83,  91,
        93,  95, 105, 127])

In [69]:
np.unique(x_train)

array([  0,   1,  22,  29,  36,  43,  50,  57,  64,  71,  78,  85,  92,
        99, 106, 113, 120, 127, 128, 134, 141, 148, 155, 256])

#XGBoost

In [70]:
!pip install xgboost

In [71]:
#model with no training data

import xgboost as xgb
model=xgb.XGBRegressor()
model=model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)])

pred_claim=model.predict(features)

[0]	validation_0-rmse:29.79117	validation_1-rmse:132.22529
[1]	validation_0-rmse:23.18322	validation_1-rmse:126.08460
[2]	validation_0-rmse:18.32819	validation_1-rmse:121.56383
[3]	validation_0-rmse:14.60309	validation_1-rmse:117.31858
[4]	validation_0-rmse:11.87017	validation_1-rmse:114.31432
[5]	validation_0-rmse:9.50770	validation_1-rmse:111.16963
[6]	validation_0-rmse:7.90816	validation_1-rmse:109.15510
[7]	validation_0-rmse:6.69392	validation_1-rmse:107.62231
[8]	validation_0-rmse:5.63266	validation_1-rmse:106.64794
[9]	validation_0-rmse:4.66763	validation_1-rmse:106.01137
[10]	validation_0-rmse:3.96424	validation_1-rmse:105.48545
[11]	validation_0-rmse:3.41074	validation_1-rmse:105.06352
[12]	validation_0-rmse:2.89121	validation_1-rmse:104.73866
[13]	validation_0-rmse:2.51292	validation_1-rmse:104.50301
[14]	validation_0-rmse:2.21366	validation_1-rmse:104.30786
[15]	validation_0-rmse:1.94412	validation_1-rmse:104.14572
[16]	validation_0-rmse:1.72494	validation_1-rmse:104.01062
[1

In [72]:
def wmape(actual, forecast):
    # make a series called mape
    se_mape = abs(actual-forecast)/actual

    # get a float of the sum of the actual
    ft_actual_sum = actual.sum()

    # get a series of the multiple of the actual & the mape
    se_actual_prod_mape = actual * se_mape

    # summate the prod of the actual and the mape
    ft_actual_prod_mape_sum = se_actual_prod_mape.sum()

    # float: wmape of forecast
    ft_wmape_forecast = ft_actual_prod_mape_sum / ft_actual_sum

    # return a float
    return ft_wmape_forecast

In [73]:
wmape(target, pred_claim)

0.9882521097403598